In [26]:
import numpy as np
import matplotlib.pyplot as plt
import copy

import csv

class Struct:
    pass

In [27]:
class MyRVNS:
    def shake(self, x, k, probdata):
        
        y = copy.deepcopy(x)
        r = np.random.permutation(probdata.n)
        
        if k == 1:             # exchange two random positions
            y.solution[r[0]] = x.solution[r[1]]
            y.solution[r[1]] = x.solution[r[0]]        
        elif k == 2:           # exchange three random positions
            y.solution[r[0]] = x.solution[r[1]]
            y.solution[r[1]] = x.solution[r[2]]
            y.solution[r[2]] = x.solution[r[0]]
        elif k == 3:           # shift positions     
            z = y.solution.pop(r[0])
            y.solution.insert(r[1],z)
        
        return y

    def neighborhoodChange(self, x, y, k):
        
        if y.fitness < x.fitness:
            x = copy.deepcopy(y)
            k = 1
        else:
            k += 1
            
        return x, k

In [28]:
class MyProblem:
    def fobj(self, x, probdata):
        # x.solution é um vetor com os custos de manutenção da i-esima maquina
        # é só somar o vetor que temos a solução
        fitness = np.sum(x.solution)
        
        x.fitness = fitness
        print(fitness)
        return x


    def sol_inicial(self, probdata, apply_constructive_heuristic):
    
        '''  
        Modelou-se uma solução x como a sequência de manutenções atribuídas em um vetor. exemplo
        
            m1 m2 ... m500
        x = [2 0 ... 1]

            nesse exemplo, a maquina m1 executa a manutenção que tem custo 2, a maquina 2 manutenção que tem custo 1, ...
        '''
        
        if apply_constructive_heuristic == False:        
            # Constrói solução inicial aleatoriamente
            x = Struct()
            x.solution = np.random.randint(0, 2 + 1, size=(probdata.n))
        
        else:
            # Constrói solução inicial usando uma heurística construtiva
            x = Struct()
            x.solution = []
            job = np.argsort(probdata.c.var(axis=0))    # tarefas ordenadas de acordo com a variância dos custos
            for tarefa in job[::-1]:        
                agent = np.argmin(probdata.c[:,tarefa]) # atribui as tarefas em ordem decrescente de variância ao agente de menor custo
                x.solution.insert(agent,tarefa)
            
        return x

    def probdef(self):
        n = 500 # numero de equipamentos

        # extrai os dados do CSV e salva na matriz
        equip_db = np.zeros(shape=(n, 4))

        with open('../arquivos_tc/EquipDB.csv', newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')

            for idx, row in enumerate(spamreader):
                row_array = ', '.join(row).split(',')
                data_array = []
                
                for item in row_array:
                    data_array.append(float(item))

                equip_db[idx] = np.array(data_array)

        # agora le o arquivo mpdb
        mpdb = np.zeros(shape=(3, 3))

        # custos do problema sao a ultima coluna do mpdb
        c = np.zeros(3)

        with open('../arquivos_tc/MPDB.csv', newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')

            for idx, row in enumerate(spamreader):
                row_array = ', '.join(row).split(',')
                data_array = []
                
                for item in row_array:
                    data_array.append(float(item))

                mpdb[idx] = np.array(data_array)
                c[idx] = data_array[len(data_array) - 1]

        probdata = Struct()
        probdata.equip_db = equip_db
        probdata.mpdb = mpdb
        probdata.c = c
        probdata.n = n
            
        return probdata

In [ ]:
myProblem = MyProblem()
myRVNS = MyRVNS()

num_sol_avaliadas = 0
max_num_sol_avaliadas = 10000
kmax = 3 # Número de estruturas de vizinhanças definidas

# salva os dados do problema 
probdata = myProblem.probdef()

print(probdata.equip_db)
print(probdata.c)

# gera uma solução inicial aleatória
x = myProblem.sol_inicial(probdata, apply_constructive_heuristic=False)

# Avalia solução inicial
x = myProblem.fobj(x, probdata)
num_sol_avaliadas += 1

print("\n------Solução inicial -----\n")
print(x.solution)
print("\n------Custo da solução inicial -----\n")
print(x.fitness)

# agora é definir os shake e vizinhanças do VNS para ele achar a solução ótima

# Armazena dados para plot
historico = Struct()
historico.sol = []
historico.fit = []
historico.sol.append(x.solution)
historico.fit.append(x.fitness)

# Ciclo iterativo do método
while num_sol_avaliadas < max_num_sol_avaliadas:
    
    k = 1
    while k <= kmax:
        
        # Gera uma solução candidata na k-ésima vizinhança de x        
        y = myRVNS.shake(x,k,probdata)
        y = myProblem.fobj(y,probdata)
        num_sol_avaliadas += 1
        
        # Atualiza solução corrente e estrutura de vizinhança (se necessário)
        x,k = myRVNS.neighborhoodChange(x,y,k)
        
        # Armazena dados para plot
        historico.sol.append(x.solution)
        historico.fit.append(x.fitness)


print('\n--- MELHOR SOLUÇÃO ENCONTRADA ---\n')
print('Sequência de tarefas atribuídas aos agentes:\n')
print('x = {}\n'.format(x.solution))
print('fitness(x) = {:.1f}\n'.format(x.fitness))

plt.figure()
s = len(historico.fit)
plt.plot(np.linspace(0,s-1,s),historico.fit,'k-')
plt.title('Evolução da qualidade da solução');
plt.xlabel('Número de avaliações');
plt.ylabel('fitness(x)');
plt.show()

[[  1.   2.   1.   5.]
 [  2.   1.   2.  10.]
 [  3.   9.   2.   9.]
 ...
 [498.   2.   2.   2.]
 [499.  32.   2.   7.]
 [500.  14.   3.   2.]]
[0. 1. 2.]
516

------Solução inicial -----

[1 0 2 0 1 2 0 1 0 2 0 0 0 1 1 2 0 0 2 1 1 2 0 1 1 2 2 2 1 2 1 1 1 1 0 0 2
 2 2 0 2 2 2 1 2 0 1 2 1 1 0 2 0 0 0 2 0 0 1 2 0 2 1 2 0 0 1 2 0 1 2 2 0 2
 2 2 1 1 0 0 2 1 1 1 1 1 1 0 2 1 1 1 2 0 2 0 2 1 1 1 0 0 1 2 0 2 0 2 2 1 0
 1 1 2 2 0 2 2 1 1 2 2 2 1 0 2 0 0 0 1 0 1 0 1 2 0 1 0 1 1 2 0 2 1 1 1 1 2
 2 0 0 0 1 2 1 1 1 2 0 1 0 0 2 0 1 1 1 1 2 1 0 0 2 0 1 2 2 0 1 0 1 1 0 2 1
 2 0 1 0 1 2 2 2 1 2 1 0 2 2 0 1 0 1 2 1 2 0 1 1 1 2 0 2 0 0 2 0 2 1 2 0 2
 2 1 2 0 1 1 1 1 1 2 0 2 2 0 1 0 2 2 0 1 1 0 2 0 1 0 0 0 1 2 0 2 1 0 1 1 0
 0 1 1 2 2 1 0 2 1 2 2 0 2 2 1 2 1 2 0 0 0 2 0 2 1 0 1 0 0 2 0 1 0 2 1 1 2
 1 0 0 2 2 2 2 0 0 2 1 0 1 0 0 1 0 1 0 2 0 0 2 0 1 0 0 1 1 0 0 2 0 2 0 0 0
 1 2 1 1 1 1 0 1 1 2 1 1 2 2 0 0 0 1 1 0 2 2 2 2 0 2 1 2 1 2 1 2 2 0 1 0 2
 0 2 1 1 1 0 1 1 2 0 2 2 1 2 2 1 0 1 2 0 1 2 2 2 2 2 1 2 1 0 